# High-level Keras (TF) Example

In [1]:
import os
import sys
import numpy as np
os.environ['KERAS_BACKEND'] = "tensorflow"
import keras as K
import tensorflow
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, Dropout
import json
from common.params import *
from common.utils import *

Using TensorFlow backend.


In [2]:
# channels_last is faster
K.backend.set_image_data_format('channels_last')

In [3]:
print("OS: ", sys.platform)
print("Python: ", sys.version)
print("Keras: ", K.__version__)
print("Numpy: ", np.__version__)
print("Tensorflow: ", tensorflow.__version__)
print(K.backend.backend())
# Channels should be last (otherwise slow)
print(K.backend.image_data_format())
print("GPU: ", get_gpu_name())

OS:  linux
Python:  3.6.4 |Anaconda, Inc.| (default, Jan 16 2018, 18:10:19) 
[GCC 7.2.0]
Keras:  2.1.4
Numpy:  1.14.0
Tensorflow:  1.6.0-rc0
tensorflow
channels_last
GPU:  ['Tesla M60']


In [4]:
os.environ['TF_ENABLE_WINOGRAD_NONFUSED'] = '1'

In [5]:
def create_symbol():
    model = Sequential()
    
    model.add(Conv2D(50, kernel_size=(3, 3), padding='same', activation='relu', input_shape=(32, 32, 3)))
    model.add(Conv2D(50, kernel_size=(3, 3), padding='same', activation='relu'))    
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(Dropout(0.25))
    
    model.add(Conv2D(100, kernel_size=(3, 3), padding='same', activation='relu'))
    model.add(Conv2D(100, kernel_size=(3, 3), padding='same', activation='relu'))    
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(Dropout(0.25))
        
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(N_CLASSES, activation='softmax'))
    return model

In [6]:
def init_model(m):
    m.compile(
        loss = "categorical_crossentropy",
        optimizer = K.optimizers.SGD(LR, MOMENTUM),
        metrics = ['accuracy'])
    return m

In [7]:
%%time
# Data into format for library
x_train, x_test, y_train, y_test = cifar_for_library(channel_first=False, one_hot=True)
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)
print(x_train.dtype, x_test.dtype, y_train.dtype, y_test.dtype)

Preparing train set...
Preparing test set...
(50000, 32, 32, 3) (10000, 32, 32, 3) (50000, 10) (10000, 10)
float32 float32 int32 int32
CPU times: user 801 ms, sys: 574 ms, total: 1.37 s
Wall time: 1.37 s


In [8]:
%%time
# Load symbol
sym = create_symbol()

CPU times: user 113 ms, sys: 5.07 ms, total: 118 ms
Wall time: 118 ms


In [9]:
%%time
# Initialise model
model = init_model(sym)

CPU times: user 28.3 ms, sys: 101 µs, total: 28.5 ms
Wall time: 27.8 ms


In [10]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 50)        1400      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 50)        22550     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 50)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 16, 16, 50)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 16, 16, 100)       45100     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 16, 16, 100)       90100     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 8, 8, 100)         0         
__________

In [11]:
results=dict()
with Timer() as t:
    # Train model
    model.fit(x_train,
              y_train,
              batch_size=BATCHSIZE,
              epochs=EPOCHS,
              verbose=1)
print('Training took %.03f sec.' % t.interval)
results['training duration']=t.interval

Epoch 1/10
50000/50000 [==============================] - 20s 393us/step - loss: 1.8128 - acc: 0.3406
Epoch 2/10
50000/50000 [==============================] - 16s 323us/step - loss: 1.3624 - acc: 0.5064
Epoch 3/10
50000/50000 [==============================] - 16s 322us/step - loss: 1.1369 - acc: 0.5918
Epoch 4/10
50000/50000 [==============================] - 16s 321us/step - loss: 0.9797 - acc: 0.6516
Epoch 5/10
50000/50000 [==============================] - 16s 322us/step - loss: 0.8585 - acc: 0.6949
Epoch 6/10
50000/50000 [==============================] - 16s 319us/step - loss: 0.7728 - acc: 0.7281
Epoch 7/10
50000/50000 [==============================] - 16s 320us/step - loss: 0.7022 - acc: 0.7529
Epoch 8/10
50000/50000 [==============================] - 16s 321us/step - loss: 0.6440 - acc: 0.7734
Epoch 9/10
50000/50000 [==============================] - 16s 320us/step - loss: 0.5941 - acc: 0.7903
Epoch 10/10
50000/50000 [==============================] - 16s 321us/step - loss: 

In [12]:
%%time
y_guess = model.predict(x_test, batch_size=BATCHSIZE)
y_guess = np.argmax(y_guess, axis=-1)
y_truth = np.argmax(y_test, axis=-1)

CPU times: user 443 ms, sys: 64.4 ms, total: 507 ms
Wall time: 1 s


In [13]:
acc=sum(y_guess == y_truth)/float(len(y_guess))
print("Accuracy: ", acc)
results['accuracy']=t.interval

Accuracy:  0.776


In [14]:
with open('keras_tf_cnn_{}.json'.format(get_gpu_name()[0].split(' ')[1].split('-')[0]), 'w') as f:
    json.dump(results, f)